In [1]:
import os
import glob
from keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow import keras
from pprint import pprint
# from tensorflow.keras.utils import print_summary
from keras_video import VideoFrameGenerator
from resnet_convlstm import ResNet
from datetime import datetime as dt

Using TensorFlow backend.


In [2]:
# use sub directories names as classes
from tensorflow_core.python.keras.optimizer_v2.adam import Adam
classes = [i.split(os.path.sep)[1] for i in glob.glob('D:/Datasets/CelebFOM_dataset/*')]
classes.sort()

In [3]:
# some global params
SIZE = (128, 128)
CHANNELS = 3
NBFRAME = 5
BS = 8

In [4]:
# pattern to get videos and classes
glob_pattern='D:/Datasets/CelebFOM_dataset/{classname}/*/*.mp4'
# glob_pattern='D:/Datasets/CelebFOM_dataset/real/*/*.mp4'
# pprint(glob.glob(glob_pattern))

In [5]:
# for data augmentation
data_aug = keras.preprocessing.image.ImageDataGenerator(
    zoom_range=.1,
    horizontal_flip=True,
    rotation_range=8,
    width_shift_range=.2,
    height_shift_range=.2)
# Create video frame generator
train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=5,
    split_val=.33, 
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    transformation=data_aug,
    use_frame_cache=True)

class fake, validation count: 718, train count: 1458
class real, validation count: 179, train count: 365
Total data: 2 classes for 1823 files for train


In [6]:
class_weights={0:3.9945,1:1}
print(train.classes)

['fake', 'real']


In [7]:
# model=keras.applications.Xception(input_shape=(128,128,3),classes=2,include_top=True,weights=None)
# model.summary()
# 
# model.compile(optimizer='nadam',
#                   loss='categorical_crossentropy',  # categorical_crossentropy if multi-class classifier
#                   metrics=['accuracy'])


model=ResNet(input_shape=(5,128, 128, 3), classes=2, block='bottleneck', residual_unit='v2',
           repetitions=[3, 4, 6, 3], initial_filters=64, activation='softmax', include_top=False,
           input_tensor=None, dropout=0.25, transition_dilation_rate=(1, 1),
           initial_strides=(2, 2), initial_kernel_size=(7, 7), initial_pooling='max',
           final_pooling=None, top='classification')
model.summary()

reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 5, 128, 128, 0                                            
__________________________________________________________________________________________________
conv_lst_m2d (ConvLSTM2D)       (None, 5, 64, 64, 64 840704      input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 5, 64, 64, 64 256         conv_lst_m2d[0][0]               
__________________________________________________________________________________________________
activation (Activation)         (None, 5, 64, 64, 64 0           batc

In [8]:
# print_summary(model, line_length=150, positions=None, print_fn=None)
adam_fine = Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='binary_crossentropy',
                  optimizer=adam_fine,
                  metrics=['accuracy'])

In [9]:
# model.summary()
# print_summary(model, line_length=150, positions=None, print_fn=None)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 5, 128, 128, 0                                            
__________________________________________________________________________________________________
conv_lst_m2d (ConvLSTM2D)       (None, 5, 64, 64, 64 840704      input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 5, 64, 64, 64 256         conv_lst_m2d[0][0]               
__________________________________________________________________________________________________
activation (Activation)         (None, 5, 64, 64, 64 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [10]:
model_checkpoint_callback = ModelCheckpoint(
    filepath="XCE_celebFOM_Ebest.h5",
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)
csv_logger = CSVLogger("MI4_DF+FS_train.csv", append=True, separator=',')

In [11]:
# model.fit_generator(train)
class_weights={0:3.9945,1:1}
# model.fit_generator(generator=train,epochs=150,
#                     callbacks=callbacks_list,shuffle=False,class_weight=class_weights,verbose=1)

# total_train=240000/8 #30,000
# total_val=40000/8   #5000
model.fit(train,
          verbose=1,
          # steps_per_epoch=int(total_train/batch_size),
          epochs=50,
          # validation_data=validation_generator,
          # validation_steps = int(total_val/batch_size),
          callbacks=[model_checkpoint_callback,csv_logger],
          class_weight=class_weights)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
Epoch 1/50
      1/Unknown - 144s 144s/step

C:\ProgramData\Anaconda3\envs\tf2\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[5,8,3,3,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/conv_lst_m2d_36/TensorArrayV2Stack/TensorListStack (defined at <ipython-input-11-e9534b7059f3>:15) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[loss/dense_loss/weighted_loss/broadcast_weights/assert_broadcastable/is_valid_shape/else/_13198/has_valid_nonscalar_shape/then/_43831/has_invalid_dims/_356]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[5,8,3,3,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/conv_lst_m2d_36/TensorA